In [106]:
%%writefile package.json
{
  "name": "fastapi-react-monorepo",
  "private": true,
  "scripts": {
    "install:all": "python -m venv .venv && .venv\\Scripts\\python.exe -m pip install uv && .venv\\Scripts\\uv.exe pip install -e api && .venv\\Scripts\\python.exe -m pip install bcrypt passlib[bcrypt] python-dotenv && (cd web && npm install)",
    "seed": ".venv\\Scripts\\python.exe api/scripts/seed_user.py",
    "dev": "concurrently -n \"API,WEB\" -c \"cyan,magenta\" \".venv\\Scripts\\python.exe -m uvicorn api.app.main:app --reload --env-file .env\" \"npm --prefix web run dev\"",
    "debug": "timeout /T 3 && curl -s http://127.0.0.1:8000/api/health && echo. && curl -s -X POST -d \"username=alice&password=secret\" -H \"Content-Type: application/x-www-form-urlencoded\" http://127.0.0.1:8000/api/token"
  },
  "devDependencies": {
    "concurrently": "^8.2.2"
  }
} 

Overwriting package.json


In [107]:
%%writefile web/.gitignore
# Byte-compiled / optimized / DLL files
__pycache__/
*.py[cod]
*$py.class
.env.example
# C extensions
*.so

# Distribution / packaging
.Python
build/
develop-eggs/
dist/
downloads/
eggs/
.eggs/
lib/
lib64/
parts/
sdist/
var/
wheels/
share/python-wheels/
*.egg-info/
.installed.cfg
*.egg
MANIFEST

# PyInstaller
#  Usually these files are written by a python script from a template
#  before PyInstaller builds the exe, so as to inject date/other infos into it.
*.manifest
*.spec

# Installer logs
pip-log.txt
pip-delete-this-directory.txt

# Unit test / coverage reports
htmlcov/
.tox/
.nox/
.coverage
.coverage.*
.cache
nosetests.xml
coverage.xml
*.cover
*.py,cover
.hypothesis/
.pytest_cache/
cover/

# Translations
*.mo
*.pot

# Django stuff:
*.log
local_settings.py
db.sqlite3
db.sqlite3-journal

# Flask stuff:
instance/
.webassets-cache

# Scrapy stuff:
.scrapy

# Sphinx documentation
docs/_build/

# PyBuilder
.pybuilder/
target/

# Jupyter Notebook
.ipynb_checkpoints

# IPython
profile_default/
ipython_config.py

# pyenv
#   For a library or package, you might want to ignore these files since the code is
#   intended to run in multiple environments; otherwise, check them in:
# .python-version

# pipenv
#   According to pypa/pipenv#598, it is recommended to include Pipfile.lock in version control.
#   However, in case of collaboration, if having platform-specific dependencies or dependencies
#   having no cross-platform support, pipenv may install dependencies that don't work, or not
#   install all needed dependencies.
#Pipfile.lock

# poetry
#   Similar to Pipfile.lock, it is generally recommended to include poetry.lock in version control.
#   This is especially recommended for binary packages to ensure reproducibility, and is more
#   commonly ignored for libraries.
#   https://python-poetry.org/docs/basic-usage/#commit-your-poetrylock-file-to-version-control
#poetry.lock

# pdm
#   Similar to Pipfile.lock, it is generally recommended to include pdm.lock in version control.
#pdm.lock
#   pdm stores project-wide configurations in .pdm.toml, but it is recommended to not include it
#   in version control.
#   https://pdm.fming.dev/#use-with-ide
.pdm.toml

# PEP 582; used by e.g. github.com/David-OConnor/pyflow and github.com/pdm-project/pdm
__pypackages__/

# Celery stuff
celerybeat-schedule
celerybeat.pid

# SageMath parsed files
*.sage.py

# Environments
.env
.venv
env/
venv/
ENV/
env.bak/
venv.bak/

# Spyder project settings
.spyderproject
.spyproject

# Rope project settings
.ropeproject

# mkdocs documentation
/site

# mypy
.mypy_cache/
.dmypy.json
dmypy.json

# Pyre type checker
.pyre/

# pytype static type analyzer
.pytype/

# Cython debug symbols
cython_debug/

# PyCharm
#  JetBrains specific template is maintained in a separate JetBrains.gitignore that can
#  be found at https://github.com/github/gitignore/blob/main/Global/JetBrains.gitignore
#  and can be added to the global gitignore or merged into this file.  For a more nuclear
#  option (not recommended) you can uncomment the following to ignore the entire idea folder.
#.idea/


Overwriting web/.gitignore


In [108]:
%%writefile web/Dockerfile.railway
# Railway Frontend Dockerfile
# =========================
# This Dockerfile implements an optimized build process with proper caching:
# - Uses explicit cache paths to avoid conflicts with Railway's build system
# - Implements multi-stage build for better layer caching
# - Separates dependency installation from build for better caching
# - Works from web directory context (Railway builds from web/)

# Build stage
FROM node:18-alpine AS build

# Accept build argument for API URL
ARG VITE_API_URL
ENV VITE_API_URL=$VITE_API_URL

# Set working directory
WORKDIR /app

# Debug: Show initial environment
RUN echo "🔍 INITIAL DEBUG: Node version:" && node --version
RUN echo "🔍 INITIAL DEBUG: NPM version:" && npm --version
RUN echo "🔍 INITIAL DEBUG: Working directory:" && pwd
RUN echo "🔍 INITIAL DEBUG: Available files:" && ls -la

# Copy package files
COPY package*.json ./

# Debug: Verify package files are copied
RUN echo "🔍 PACKAGE DEBUG: Package files after copy:" && ls -la package*
RUN echo "🔍 PACKAGE DEBUG: Package.json content:" && head -20 package.json
RUN echo "🔍 PACKAGE DEBUG: Package-lock.json exists:" && test -f package-lock.json && echo "YES" || echo "NO"
RUN echo "🔍 PACKAGE DEBUG: Build script:" && cat package.json | grep -A 3 -B 3 '"build"'

# Install dependencies with better error handling
RUN echo "🔍 BUILD DEBUG: Installing dependencies..." && \
    npm ci --prefer-offline --no-audit --loglevel=error

# Debug: Verify node_modules
RUN echo "🔍 INSTALL DEBUG: Node modules created:" && ls -la node_modules | head -10
RUN echo "🔍 INSTALL DEBUG: Key packages installed:" && \
    ls node_modules/ | grep -E "(react|vite|rollup|typescript)" || echo "Key packages not found"

# Copy source code
COPY . .

# Debug: Show what files are available after source copy
RUN echo "🔍 SOURCE DEBUG: All files after source copy:" && ls -la
RUN echo "🔍 SOURCE DEBUG: Source directory structure:" && find . -type f -name "*.ts*" -o -name "*.js*" | head -20
RUN echo "🔍 SOURCE DEBUG: TypeScript config files:" && ls -la tsconfig*

# Debug environment variables
RUN echo "🔍 BUILD DEBUG: VITE_API_URL = ${VITE_API_URL}"
RUN echo "🔍 BUILD DEBUG: All ENV vars:" && env | grep -E "(VITE_|NODE_|RAILWAY_)" || true

# Pre-build TypeScript check with detailed error reporting
RUN echo "🔍 BUILD DEBUG: Running TypeScript check..." && \
    npm run type-check:verbose 2>&1 | tee typecheck.log || \
    (echo "⚠️  TypeScript check failed, but continuing with build..." && \
     echo "🔍 BUILD DEBUG: TypeScript errors:" && \
     cat typecheck.log && \
     echo "🔍 BUILD DEBUG: Attempting build anyway...")

# Build with fallback strategy
RUN echo "🔍 BUILD DEBUG: Starting build process..." && \
    (npm run build 2>&1 | tee build.log || \
     (echo "⚠️  Standard build failed, trying force build..." && \
      npm run build:force 2>&1 | tee build-force.log))

# Verify build output
RUN echo "🔍 BUILD DEBUG: Verifying build output..." && \
    ls -la dist/ && \
    echo "🔍 BUILD DEBUG: Build files:" && \
    find dist -type f -name "*.js" -o -name "*.css" -o -name "*.html" | head -10

# Check for API URL in built files
RUN echo "🔍 BUILD DEBUG: Checking built files for API URL..." && \
    find dist -name "*.js" -exec grep -l "${VITE_API_URL:-fastapi-production-1d13.up.railway.app}" {} \; || \
    echo "🔍 BUILD DEBUG: API URL not found in built files"

# Sample content from built JS files
RUN echo "🔍 BUILD DEBUG: Sample content from built JS files:" && \
    find dist -name "*.js" | head -1 | xargs head -50 || \
    echo "🔍 BUILD DEBUG: No JS files found"

# Search for VITE_API_URL in built files
RUN echo "🔍 BUILD DEBUG: Searching for VITE_API_URL in built files:" && \
    find dist -name "*.js" -exec grep -H "VITE_API_URL" {} \; || \
    echo "🔍 BUILD DEBUG: VITE_API_URL not found in built files"

# Final verification that dist exists
RUN echo "🔍 BUILD DEBUG: Final verification..." && \
    test -d dist && echo "✅ dist directory exists" || \
    (echo "❌ dist directory missing - build failed!" && exit 1)

# Production stage
FROM nginx:alpine

# Copy built assets from build stage
COPY --from=build /app/dist ./dist

# Copy nginx configuration
COPY nginx.conf /etc/nginx/nginx.conf

# Expose port
EXPOSE 80

# Health check
HEALTHCHECK --interval=30s --timeout=10s --start-period=5s --retries=3 \
  CMD wget --no-verbose --tries=1 --spider http://localhost:80/ || exit 1

# Start nginx
CMD ["nginx", "-g", "daemon off;"] 


Overwriting web/Dockerfile.railway


In [109]:
%%writefile web/eslint.config.js
import js from '@eslint/js'
import globals from 'globals'
import reactHooks from 'eslint-plugin-react-hooks'
import reactRefresh from 'eslint-plugin-react-refresh'
import tseslint from 'typescript-eslint'

export default tseslint.config(
  { ignores: ['dist'] },
  {
    extends: [js.configs.recommended, ...tseslint.configs.recommended],
    files: ['**/*.{ts,tsx}'],
    languageOptions: {
      ecmaVersion: 2020,
      globals: globals.browser,
    },
    plugins: {
      'react-hooks': reactHooks,
      'react-refresh': reactRefresh,
    },
    rules: {
      ...reactHooks.configs.recommended.rules,
      'react-refresh/only-export-components': [
        'warn',
        { allowConstantExport: true },
      ],
    },
  },
)


Overwriting web/eslint.config.js


In [110]:
%%writefile web/index.html
<!doctype html>
<html lang="en">
  <head>
    <meta charset="UTF-8" />
    <link rel="icon" type="image/svg+xml" href="/vite.svg" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0" />
    <title>Vite + React + TS</title>
  </head>
  <body>
    <div id="root"></div>
    <script type="module" src="/src/main.tsx"></script>
  </body>
</html>


Overwriting web/index.html


In [111]:
%%writefile web/nixpacks.toml
[phases.setup]
nixPkgs = ["nodejs-18_x", "npm-9_x"]

[phases.install]
cmds = [
  "npm ci --prefer-offline --no-audit --loglevel=error"
]

[phases.build]
cmds = [
  "npm run build"
]

[start]
cmd = "npx serve -s dist -l $PORT --no-clipboard --no-port-switching"

[variables]
NODE_ENV = "production"
NPM_CONFIG_PRODUCTION = "false" 

Overwriting web/nixpacks.toml


In [112]:
%%writefile web/package.json
{
  "name": "vite-react-typescript-starter",
  "private": true,
  "version": "0.0.0",
  "type": "module",
  "scripts": {
    "dev": "vite",
    "build": "vite build",
    "type-check": "tsc -p tsconfig.app.json --noEmit",
    "lint": "eslint .",
    "preview": "vite preview"
  },
  "dependencies": {
    "react": "^18.2.0",
    "react-dom": "^18.2.0",
    "axios": "^1.6.7"
  },
  "devDependencies": {
    "@types/node": "^20.11.24",
    "@types/react": "^18.2.61",
    "@types/react-dom": "^18.2.19",
    "@vitejs/plugin-react": "^4.2.1",
    "eslint": "^8.57.0",
    "eslint-plugin-react-hooks": "^4.6.0",
    "eslint-plugin-react-refresh": "^0.4.5",
    "typescript": "^5.3.3",
    "@typescript-eslint/eslint-plugin": "^7.1.0",
    "@typescript-eslint/parser": "^7.1.0",
    "vite": "^5.1.4",
    "rollup": "^4.16.0"
  }
}


Overwriting web/package.json


In [113]:
%%writefile web/railway.alternative.json
{
  "$schema": "https://railway.app/railway.schema.json",
  "build": {
    "builder": "DOCKERFILE",
    "dockerfilePath": "Dockerfile.railway"
  },
  "deploy": {
    "startCommand": "npx serve -s dist -l $PORT --no-clipboard --no-port-switching",
    "restartPolicyType": "ON_FAILURE",
    "restartPolicyMaxRetries": 10
  }
} 

Overwriting web/railway.alternative.json


In [114]:
%%writefile web/railway.json
{
  "$schema": "https://railway.app/railway.schema.json",
  "build": {
    "builder": "DOCKERFILE",
    "dockerfilePath": "Dockerfile.railway",
    "buildArgs": {
      "VITE_API_URL": "${{ VITE_API_URL }}"
    }
  },
  "deploy": {
    "restartPolicyType": "ON_FAILURE",
    "restartPolicyMaxRetries": 10
  }
}



Overwriting web/railway.json


In [115]:
%%writefile web/tsconfig.app.json
{
  "compilerOptions": {
    "composite": true,
    "module": "ESNext",
    "moduleResolution": "Node",
    "allowJs": true,
    "esModuleInterop": true,
    "resolveJsonModule": true,
    "lib": ["ESNext", "DOM"],
    "types": ["node", "vite/client"],
    "skipLibCheck": true,
    "noEmit": true
  },
  "include": ["vite.config.ts"]
} 


Overwriting web/tsconfig.app.json


In [116]:
%%writefile web/tsconfig.json
{
  "files": [],
  "references": [
    { "path": "./tsconfig.app.json" },
    { "path": "./tsconfig.node.json" }
  ]
}


Overwriting web/tsconfig.json


In [117]:
%%writefile web/tsconfig.node.json
{
  "compilerOptions": {
    "composite": true,
    "module": "ESNext",
    "moduleResolution": "Node",
    "allowJs": true,
    "esModuleInterop": true,
    "resolveJsonModule": true,
    "lib": ["ESNext"],
    "types": ["node"]
  },
  "include": ["vite.config.ts"]
} 

Overwriting web/tsconfig.node.json


In [118]:
%%writefile web/vite.config.ts
import { defineConfig, loadEnv } from 'vite'
import react from '@vitejs/plugin-react'

// https://vite.dev/config/
export default defineConfig(({ mode }) => {
  const env = loadEnv(mode, process.cwd(), '')
  const raw = env.VITE_API_URL || 
              (mode === 'production'
                ? 'https://fastapi-production-1d13.up.railway.app'
                : 'http://127.0.0.1:8000')

  /** Prepend https:// if the scheme is missing */
  const API_URL = /^https?:\/\//.test(raw) ? raw.replace(/\/+$/, '') : `https://${raw}`

  // Final sanity-check – bail if still invalid
  if (!/^https?:\/\/[^/]+$/.test(API_URL)) {
    throw new Error(`VITE_API_URL must be an absolute URL – got "${API_URL}"`);
  }

  console.log('🔍 Vite Config Debug:')
  console.log('Mode:', mode)
  console.log('Raw API_URL:', raw)
  console.log('Normalized API_URL:', API_URL)
  console.log('All VITE_ env vars:', Object.keys(env).filter(key => key.startsWith('VITE_')))

  return {
    plugins: [react()],
    /** Inject BOTH names so legacy code keeps working */
    define: {
      __API_URL__: JSON.stringify(API_URL),
      'import.meta.env.VITE_API_URL': JSON.stringify(API_URL)
    },
    server: {
      host: '0.0.0.0',
      port: 5173,
      proxy: {
        '/api': {
          target: API_URL,
          changeOrigin: true,
          secure: false,
          rewrite: (path) => path, // Keep the /api prefix
          configure: (proxy, _options) => {
            proxy.on('error', (err, _req, _res) => {
              console.log('🔍 Proxy Error:', err)
            })
            proxy.on('proxyReq', (proxyReq, req, _res) => {
              console.log('🔍 Proxy Request:', req.method, req.url, '-> ', proxyReq.path)
            })
            proxy.on('proxyRes', (proxyRes, req, _res) => {
              console.log('🔍 Proxy Response:', proxyRes.statusCode, req.url)
            })
          }
        }
      }
    },
    build: {
      outDir: 'dist',
      assetsDir: 'assets',
      sourcemap: false,
      chunkSizeWarningLimit: 1000,
      rollupOptions: {
        output: {
          manualChunks: {
            vendor: ['react', 'react-dom']
          }
        }
      }
    },
    esbuild: {
      logOverride: { 
        'this-is-undefined-in-esm': 'silent'
      },
      target: 'es2020',
      keepNames: true
    }
  }
})


Overwriting web/vite.config.ts


In [119]:
%%writefile web/src/vite-env.d.ts
/// <reference types="vite/client" />

declare module '*.svg' {
  const svgUrl: string;
  export default svgUrl;
}


Overwriting web/src/vite-env.d.ts


In [120]:
%%writefile web/src/contexts/AuthContext.tsx
import React from 'react';
import { createContext, useContext, useState, ReactNode } from 'react';

interface AuthCtx {
  token: string | null;
  verified: boolean;
  login: (t: string) => void;
  logout: () => void;
  markVerified: () => void;
  invalidate: () => void;
}

const Ctx = createContext<AuthCtx | undefined>(undefined);

export const AuthProvider: React.FC<{ children: ReactNode }> = ({ children }) => {
  const [token, setToken] = useState<string | null>(() => localStorage.getItem('jwt'));
  const [verified, setVerified] = useState(false);

  const login = (t: string) => {
    setToken(t);
    setVerified(false);              // must re-check with server
    localStorage.setItem('jwt', t);
  };

  const logout = () => {
    setToken(null);
    setVerified(false);
    localStorage.removeItem('jwt');
  };

  const markVerified = () => setVerified(true);
  const invalidate   = () => logout();

  return (
    <Ctx.Provider value={{ token, verified, login, logout, markVerified, invalidate }}>
      {children}
    </Ctx.Provider>
  );
};

export const useAuth = () => {
  const ctx = useContext(Ctx);
  if (!ctx) throw new Error('useAuth must be inside AuthProvider');
  return ctx;
};

// No hooks – safe for axios interceptor
export const getStoredToken = () => localStorage.getItem('jwt'); 




Overwriting web/src/contexts/AuthContext.tsx


In [121]:
%%writefile web/src/pages/Login.tsx
import { FormEvent, useState } from 'react'
import { useAuth } from '../contexts/AuthContext'
import { api } from '../api/axios'

export default function Login() {
  const { login } = useAuth()
  const [user, setUser] = useState('')
  const [pwd, setPwd] = useState('')
  const [error, setError] = useState('')

  const handleSubmit = async (e: FormEvent) => {
    e.preventDefault()
    try {
      const form = new URLSearchParams()
      form.append('username', user)
      form.append('password', pwd)
      const { data } = await api.post('/token', form, {
        headers: { 'Content-Type': 'application/x-www-form-urlencoded' },
      })
      login(data.access_token)
    } catch (err) {
      setError('Invalid credentials')
    }
  }

  return (
    <form onSubmit={handleSubmit} className="login-form">
      <h2>Login</h2>
      <div className="form-group">
        <input
          value={user}
          onChange={e => setUser(e.target.value)}
          placeholder="Username"
          className="form-input"
        />
      </div>
      <div className="form-group">
        <input
          type="password"
          value={pwd}
          onChange={e => setPwd(e.target.value)}
          placeholder="Password"
          className="form-input"
        />
      </div>
      <button type="submit" className="login-button">Login</button>
      {error && <p className="error-message">{error}</p>}
    </form>
  )
} 


Overwriting web/src/pages/Login.tsx


In [122]:
%%writefile web/src/api/axios.ts
import axios, { InternalAxiosRequestConfig } from 'axios';
import { getStoredToken } from '../contexts/AuthContext';

declare const __API_URL__: string;  // Declare the injected constant

const root = __API_URL__;              // Now guaranteed absolute
export const api = axios.create({ baseURL: `${root}/api` });

api.interceptors.request.use((cfg: InternalAxiosRequestConfig) => {
  const t = getStoredToken();
  cfg.headers = cfg.headers || {};
  if (t) cfg.headers.Authorization = `Bearer ${t}`;
  return cfg;
}); 



Overwriting web/src/api/axios.ts


In [123]:
%%writefile web/src/App.css
#root {
  max-width: 1280px;
  margin: 0 auto;
  padding: 2rem;
  text-align: center;
}

.logo {
  height: 6em;
  padding: 1.5em;
  will-change: filter;
  transition: filter 300ms;
}
.logo:hover {
  filter: drop-shadow(0 0 2em #646cffaa);
}
.logo.react:hover {
  filter: drop-shadow(0 0 2em #61dafbaa);
}

@keyframes logo-spin {
  from {
    transform: rotate(0deg);
  }
  to {
    transform: rotate(360deg);
  }
}

@media (prefers-reduced-motion: no-preference) {
  a:nth-of-type(2) .logo {
    animation: logo-spin infinite 20s linear;
  }
}

.card {
  padding: 2em;
}

.read-the-docs {
  color: #888;
}


Overwriting web/src/App.css


In [124]:
%%writefile web/src/App.tsx
import React from 'react';
import { useState, useEffect } from 'react';
import { useAuth } from './contexts/AuthContext';
import Login from './pages/Login';
// @ts-ignore
import reactLogo from './assets/react.svg';
import './App.css';

interface ImportMetaEnv {
  readonly DEV: boolean;
  readonly PROD: boolean;
}

interface ImportMeta {
  readonly env: ImportMetaEnv;
}

declare const __API_URL__: string;
const buildUrl = (e: string) => import.meta.env.DEV ? (e.startsWith('/api') ? e : `/api${e}`) : `${__API_URL__}${e}`;

interface PredictionResponse {
  prediction: string;
  confidence: number;
  input_received: { count: number };
}

export default function App() {
  const { token, verified, markVerified, invalidate, logout } = useAuth();
  const [count, setCount] = useState(0);
  const [apiMessage, setApiMessage] = useState('');
  const [error, setError] = useState('');
  const [prediction, setPrediction] = useState<PredictionResponse | null>(null);

  /* ---- validate stored JWT once on mount --------------------------------- */
  useEffect(() => {
    if (!token || verified) return;                 // nothing to do
    (async () => {
      try {
        const res = await fetch(buildUrl('/api/hello'), {
          headers: { Authorization: `Bearer ${token}` },
        });
        if (!res.ok) throw new Error('token invalid');
        markVerified();
      } catch {
        invalidate();                               // wipes bad token
      }
    })();
  }, [token, verified, markVerified, invalidate]);

  /* ---- helper to call JSON endpoints ------------------------------------- */
  const fetchJson = async <T,>(endpoint: string, init?: RequestInit): Promise<T> => {
    const headers = new Headers(init?.headers);
    headers.set('Content-Type', 'application/json');
    if (token) headers.set('Authorization', `Bearer ${token}`);
    const res = await fetch(buildUrl(endpoint), { ...init, headers });
    if (!res.ok) throw new Error(`${res.status} ${res.statusText}`);
    return res.json() as Promise<T>;
  };

  /* ---- grab greeting after verification ---------------------------------- */
  useEffect(() => {
    if (!verified) return;
    fetchJson<{ message: string }>('/api/hello')
      .then(({ message }) => setApiMessage(message))
      .catch(err => setError(err.message));
  }, [verified]);

  /* ---- RENDER ------------------------------------------------------------ */
  if (!token || !verified) return <Login />;   // ← always initial view

  return (
    <>
      <div>
        <img src="/vite.svg" className="logo" alt="Vite logo" />
        <a href="https://react.dev" target="_blank" rel="noreferrer">
          <img src={reactLogo} className="logo react" alt="React logo" />
        </a>
      </div>

      <h1>FastAPI + React + Railway</h1>

      <div className="card">
        <h2>🔗 API Connection</h2>
        {error && <p style={{ color: 'red' }}>❌ {error}</p>}
        {apiMessage && <p style={{ color: 'green' }}>✅ {apiMessage}</p>}

        <button onClick={logout}>Logout</button>
        <button onClick={() => setCount(c => c + 1)}>Count {count}</button>
        <button
          onClick={() =>
            fetchJson<PredictionResponse>('/api/predict', {
              method: 'POST',
              body: JSON.stringify({ data: { count } }),
            })
              .then(r => {
                setPrediction(r);
                setError('');
              })
              .catch(e => {
                setError(e.message);
                setPrediction(null);
              })
          }
        >
          Test Prediction
        </button>

        {prediction && (
          <div style={{ marginTop: '1rem', padding: '1rem', border: '1px solid #ddd', borderRadius: '4px' }}>
            <h3>Prediction Results</h3>
            <p>Input count: {prediction.input_received.count}</p>
            <p>Prediction: {prediction.prediction}</p>
            <p>Confidence: {(prediction.confidence * 100).toFixed(1)}%</p>
          </div>
        )}
      </div>
    </>
  );
}



Overwriting web/src/App.tsx


In [125]:
%%writefile web/src/index.css
:root {
  font-family: system-ui, Avenir, Helvetica, Arial, sans-serif;
  line-height: 1.5;
  font-weight: 400;

  color-scheme: light dark;
  color: rgba(255, 255, 255, 0.87);
  background-color: #242424;

  font-synthesis: none;
  text-rendering: optimizeLegibility;
  -webkit-font-smoothing: antialiased;
  -moz-osx-font-smoothing: grayscale;
}

a {
  font-weight: 500;
  color: #646cff;
  text-decoration: inherit;
}
a:hover {
  color: #535bf2;
}

body {
  margin: 0;
  display: flex;
  place-items: center;
  min-width: 320px;
  min-height: 100vh;
}

h1 {
  font-size: 3.2em;
  line-height: 1.1;
}

button {
  border-radius: 8px;
  border: 1px solid transparent;
  padding: 0.6em 1.2em;
  font-size: 1em;
  font-weight: 500;
  font-family: inherit;
  background-color: #1a1a1a;
  cursor: pointer;
  transition: border-color 0.25s;
}
button:hover {
  border-color: #646cff;
}
button:focus,
button:focus-visible {
  outline: 4px auto -webkit-focus-ring-color;
}

/* Login Form Styles */
.login-form {
  max-width: 400px;
  margin: 2rem auto;
  padding: 2rem;
  border-radius: 8px;
  box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1);
  background-color: white;
}

.login-form h2 {
  margin-bottom: 1.5rem;
  text-align: center;
  color: #333;
}

.form-group {
  margin-bottom: 1rem;
}

.form-input {
  width: 100%;
  padding: 0.75rem;
  border: 1px solid #ddd;
  border-radius: 4px;
  font-size: 1rem;
}

.form-input:focus {
  outline: none;
  border-color: #4a90e2;
  box-shadow: 0 0 0 2px rgba(74, 144, 226, 0.2);
}

.login-button {
  width: 100%;
  padding: 0.75rem;
  background-color: #4a90e2;
  color: white;
  border: none;
  border-radius: 4px;
  font-size: 1rem;
  cursor: pointer;
  transition: background-color 0.2s;
}

.login-button:hover {
  background-color: #357abd;
}

.error-message {
  color: #dc3545;
  margin-top: 1rem;
  text-align: center;
  font-size: 0.875rem;
}

@media (prefers-color-scheme: light) {
  :root {
    color: #213547;
    background-color: #ffffff;
  }
  a:hover {
    color: #747bff;
  }
  button {
    background-color: #f9f9f9;
  }
}


Overwriting web/src/index.css


In [126]:
%%writefile web/src/main.tsx
import { StrictMode } from 'react';
import { createRoot } from 'react-dom/client';
import { AuthProvider } from './contexts/AuthContext';
import App from './App';
import './index.css';

createRoot(document.getElementById('root')!).render(
  <StrictMode>
    <AuthProvider>
      <App />
    </AuthProvider>
  </StrictMode>
);



Overwriting web/src/main.tsx


In [127]:
%%writefile web/src/vite-env.d.ts
/// <reference types="vite/client" />


Overwriting web/src/vite-env.d.ts
